In [ ]:
from os import listdir
from os.path import isfile, join
import networkx as nx
import numpy as np
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice import BasicElement
from PySpice.Spice.Netlist import Node
import helpers as h
import multiprocessing
from tqdm.notebook import tqdm
import tqdm
from joblib import Parallel, delayed
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import multiprocessing
from sklearn.utils import shuffle
import dgsd
import pickle
import netlsd
import torch
from torch.nn import functional as F
from sklearn.metrics import f1_score, accuracy_score,roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_netlsd_embeddings(train_graphs,test_graphs, train_y, test_y):
    train_embeddings_netlsd,train_y_netlsd = [],[]
    for index, g in enumerate(train_graphs):
        try:
            emb = netlsd.heat(g)
            train_embeddings_netlsd.append(emb)
            train_y_netlsd.append(train_y[index])

        except:
            print("didn't work!",index)
        if index>0 and index%1000==0:
            print("{} procssed!".format(index))
    print("TESTING GRAPHS")
    test_embeddings_netlsd,test_y_netlsd = [],[]
    for index, g in enumerate(test_graphs):
        try:
            emb = netlsd.heat(g)
            test_embeddings_netlsd.append(emb)
            test_y_netlsd.append(test_y[index])

        except:
            print("didn't work!",index)
        if index>0 and index%100==0:
            print("{} procssed!".format(index))
    clf=RandomForestClassifier(n_estimators=500,n_jobs = multiprocessing.cpu_count()-2)

    print("fitting classifier!")
    clf.fit(train_embeddings_netlsd,train_y_netlsd)
    y_pred=clf.predict(test_embeddings_netlsd)
    y = F.one_hot(torch.tensor(test_y_netlsd), num_classes = 5).numpy()
    pred = F.one_hot(torch.tensor(y_pred),num_classes = 5).numpy()
    auc = roc_auc_score(pred,y,average='weighted', multi_class='ovr')
    acc = accuracy_score(y_pred, test_y_netlsd)
    f1 = f1_score(y_pred, test_y_netlsd,average='weighted')
        
    return auc, acc,f1
def get_dgsd_embeddings(train_graphs,test_graphs, train_y, test_y):
    train_embeddings_dgsd,train_y_dgsd = [],[]
    for index, g in enumerate(train_graphs):
        try:
            emb = dgsd.DGSD().get_descriptor(g)
            train_embeddings_dgsd.append(emb)
            train_y_dgsd.append(train_y[index])
        except:
            print("didn't work!",index)
        if index>0 and index%100==0:
            print("{} procssed!".format(index))
    print("TESTING GRAPHS")
    test_embeddings_dgsd,test_y_dgsd = [],[]
    for index, g in enumerate(test_graphs):
        try:
            emb = dgsd.DGSD().get_descriptor(g)
            test_embeddings_dgsd.append(emb)
            test_y_dgsd.append(test_y[index])
        except:
            print("didn't work!",index)
        if index>0 and index%100==0:
            print("{} procssed!".format(index))
    clf=RandomForestClassifier(n_estimators=500,n_jobs = multiprocessing.cpu_count()-2)
    print("fitting classifier!")
    clf.fit(train_embeddings_dgsd,train_y_dgsd)
    y_pred=clf.predict(test_embeddings_dgsd)
#     acc = roc_auc_score(y_pred, test_y_dgsd,average='macro', multi_class='ovr')
#     print("accuracy dgsd:",acc)
    y = F.one_hot(torch.tensor(test_y_dgsd), num_classes = 5).numpy()
    pred = F.one_hot(torch.tensor(y_pred),num_classes = 5).numpy()
    auc = roc_auc_score(y, pred,average='weighted', multi_class='ovr')
    f1 = f1_score(y_pred, test_y_dgsd,average='weighted')
    acc = accuracy_score(y_pred, test_y_netlsd)
#     print("f1 score dgsd:",f1)
    return auc,acc, f1

In [ ]:
# train_data = pickle.load(open("data/combined_GC_train_x.pkl",'rb'))
# test_data = pickle.load(open("data/combined_GC_test_x.pkl",'rb'))
# train_y = pickle.load(open("data/combined_GC_train_y.pkl","rb"))
# test_y = pickle.load(open("data/combined_GC_test_y.pkl","rb"))
with open("data/kicad_github_GC.pkl",'rb') as f:
    data = pickle.load(f)
    f.close()
with open('data/kicad_github_label_mapping.pkl', 'rb') as f:
    mapping = pickle.load(f)
    f.close()

train_graphs = data['train_x']
test_graphs = data['test_x'] 
train_y = data['train_y'] 
test_y = data['test_y']


auc,acc,f1 = get_netlsd_embeddings(train_graphs,test_graphs, train_y, test_y)
print("auc: {}, acc: {}, f1:{}".format(auc,acc,f1))

In [ ]:
with open("data/ltspice_examples_GC.pkl",'rb') as f:
    data = pickle.load(f)
    f.close()
with open('data/ltspice_examples_label_mapping.pkl', 'rb') as f:
    mapping = pickle.load(f)
    f.close()

train_graphs = data['train_x']
test_graphs = data['test_x'] 
train_y = data['train_y'] 
test_y = data['test_y']


auc,acc,f1 = get_netlsd_embeddings(train_graphs,test_graphs, train_y, test_y)
print("auc: {}, acc: {}, f1:{}".format(auc,acc,f1))

In [ ]:
auc,acc,f1 = get_dgsd_embeddings(train_graphs,test_graphs, train_y, test_y)
print("auc: {}, acc: {}, f1:{}".format(auc,acc, f1))

In [ ]:
# nx.draw(train_graphs[16])